In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Inference

In [3]:
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'maximum_weight_recommendation': {'gram',
        'kilogram',
        'microgram',
        'milligram',
        'ounce',
        'pound',
        'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre',
        'cubic foot',
        'cubic inch',
        'cup',
        'decilitre',
        'fluid ounce',
        'gallon',
        'imperial gallon',
        'litre',
        'microlitre',
        'millilitre',
        'pint',
        'quart'}
}

In [4]:
%pip install torch transformers timm einops datasets bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires

In [5]:
# Initialize moondream. Change DEVICE to 'mps' if you're on an M1 Mac, or 'cpu' if you don't have a
# GPU. Note that fine-tuning on CPU will be very slow.

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

DEVICE = "cuda"
DTYPE = torch.float32 if DEVICE == "cpu" else torch.float16 # CPU doesn't support float16
MD_REVISION = "2024-07-23"

tokenizer = AutoTokenizer.from_pretrained("vikhyatk/moondream2", revision=MD_REVISION)
moondream = AutoModelForCausalLM.from_pretrained(
    "/content/drive/MyDrive/moondream-ft_large_prompt_after_10000_iterations", revision=MD_REVISION, trust_remote_code=True,
    torch_dtype=DTYPE, device_map={"": DEVICE}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

In [10]:
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset
import requests
from PIL import Image
from io import BytesIO

class CustomTestImageDataset(Dataset):
    def __init__(self, csv_file):
        # Load the CSV file into a pandas DataFrame
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        # Return the total number of samples
        return len(self.data)

    def __getitem__(self, idx):
        # Get the row corresponding to the given index
        row = self.data.iloc[idx]

        # Extract the image URL and entity name
        image_url = row['image_link']
        entity_name = row['entity_name']

        # Replace underscores in entity_name with spaces
        entity_name_cleaned = entity_name.replace('_', ' ')

        # Download and open the image in grayscale
        response = requests.get(image_url)
        img = Image.open(BytesIO(response.content)).convert('L')

        # Create the prompt using the cleaned entity name
        prompt = f"""Extract {entity_name_cleaned} from the image in the format 'x unit', where: 'x' is a float number in standard formatting.'unit' is one of the allowed units for {entity_name_cleaned} from the following list: {', '.join(entity_unit_map[entity_name])}.Ensure that the output strictly matches the format "x unit" with a space separating the number and the unit. Do not use any abbreviations, special characters, or additional text. If no valid value is found in the image, return a string "blank" """

        # Return the sample with its original index
        return {
            "index": idx,  # Keep track of the original index
            "image": img,  # Grayscale PIL image
            "qa": [
                {
                    "question": prompt,
                    "answer": "",  # No ground truth answer for test data
                }
            ]
        }

# Load the CSV file into a pandas DataFrame
test_df = pd.read_csv('part21.csv')

# Initialize the custom test dataset
test_dataset = CustomTestImageDataset('part21.csv')

# Prepare an empty list to store the predictions and their indices
predictions = []

# Perform model inference
for i, sample in tqdm(enumerate(test_dataset), total=len(test_dataset), desc="Predicting"):
    # Generate prediction using the model
    md_answer = moondream.answer_question(
        moondream.encode_image(sample['image']),
        sample['qa'][0]['question'],
        tokenizer=tokenizer,
        num_beams=4,
        no_repeat_ngram_size=5,
        early_stopping=True
    )

    # Append the prediction along with its original index
    predictions.append((sample["index"], md_answer))

    # Print for debugging or confirmation
    print('Question:', sample['qa'][0]['question'])
    print('Predicted Answer:', md_answer)

# Sort the predictions list by the original index to maintain order
predictions.sort(key=lambda x: x[0])

# Extract the sorted predictions
sorted_predictions = [pred[1] for pred in predictions]

# Add the sorted predictions as a new column in the DataFrame
test_df['entity_value'] = sorted_predictions

# Save the updated DataFrame back to the CSV file
test_df.to_csv('updated_merged_test_2.csv', index=False)

print("Predictions have been saved to 'updated_test.csv'.")

Output hidden; open in https://colab.research.google.com to view.

# Split

In [8]:
import pandas as pd

# Step 1: Read the CSV file
file_path = '/content/merged_test_2.csv'  # Replace with the path to your CSV file
df = pd.read_csv(file_path)

# Step 2: Determine the number of rows
total_rows = len(df)

# Step 3: Calculate the split points for dividing into three parts
split1 = total_rows // 3
split2 = 2 * (total_rows // 3)

# Step 4: Split the DataFrame
df_part1 = df.iloc[:split1]
df_part2 = df.iloc[split1:split2]
df_part3 = df.iloc[split2:]

# Step 5: Save each part as separate CSV files
df_part1.to_csv('part21.csv', index=False)
df_part2.to_csv('part22.csv', index=False)
df_part3.to_csv('part23.csv', index=False)

print("CSV file split into three parts successfully.")


CSV file split into three parts successfully.
